In [1]:
!pip uninstall -y transformers


In [2]:
!pip install transformers==4.36.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 32.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.2 which is incompatible.


In [3]:
!pip install -U transformers datasets scikit-learn pandas huggingface_hub



  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 19.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.2
    Uninstalling transformers-4.36.2:
      Successfully uninstalled transformers-4.36.2


In [4]:
!pip install git+https://github.com/huggingface/transformers.git



  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-imajw5re
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-imajw5re
  Resolved https://github.com/huggingface/transformers.git to commit 3165eb7c2808832d0de86c8f508d9da6b2124044
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.52.0.dev0-py3-none-any.whl size=11299668 sha256=cb906b39b514636a51077fc6f43fcfef3a281d1525b3cc0a13500fc42035f6a4
  Stored in directory: /tmp/pip-ephem-wheel-cache-7jq9n0p3/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


In [5]:
from google.colab import files
uploaded = files.upload()


Saving LLM.csv to LLM (4).csv


In [6]:
import pandas as pd

# Load data
df = pd.read_csv("LLM.csv")

# Keep only necessary columns and drop missing values
df = df[['Text', 'Label']].dropna()

# Normalize labels
df['Label'] = df['Label'].str.lower().map({'ai': 1, 'student': 0})

# Rename for consistency
df.columns = ['text', 'label']

df.head()


,text,label
0,y r u always l8 to the meetings?,0
1,The project team embraced a user-centric desig...,1
2,"i dont like dealing with risks, it's too stres...",0
3,"i dont worry about reliability, it's good enough",0
4,"i dont care about human-centered design, just ...",0


In [7]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)



In [8]:
from transformers import AutoTokenizer

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, list(train_labels))
val_dataset = CustomDataset(val_encodings, list(val_labels))


In [10]:
import transformers
print(transformers.__version__)


4.52.0.dev0


In [18]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    logging_dir="./logs",
    do_train=True,
    do_eval=True,
    logging_steps=10,
    save_steps=100,
    eval_steps=100,
    save_total_limit=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rais-s (rais-s-northeastern-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.385100
20,0.137200
30,0.094200
40,0.030800
50,0.093600
60,0.008500
70,0.005500
80,0.004400
90,0.003200
100,0.041500


TrainOutput(global_step=333, training_loss=0.024856518559694826, metrics={'train_runtime': 1827.6042, 'train_samples_per_second': 1.446, 'train_steps_per_second': 0.182, 'total_flos': 31238785047420.0, 'train_loss': 0.024856518559694826, 'epoch': 3.0})

In [20]:
trainer.evaluate()


{'eval_loss': 0.0004366487846709788,
 'eval_runtime': 22.1725,
 'eval_samples_per_second': 9.967,
 'eval_steps_per_second': 1.263,
 'epoch': 3.0}

In [21]:
from sklearn.metrics import classification_report
import numpy as np

preds = trainer.predict(val_dataset)
y_true = list(val_labels)
y_pred = np.argmax(preds.predictions, axis=1)

print(classification_report(y_true, y_pred, target_names=["Student", "AI"]))


              precision    recall  f1-score   support

     Student       1.00      1.00      1.00       110
          AI       1.00      1.00      1.00       111

    accuracy                           1.00       221
   macro avg       1.00      1.00      1.00       221
weighted avg       1.00      1.00      1.00       221



In [22]:
val_texts = val_texts.reset_index(drop=True)

for i in range(len(y_pred)):
    if y_pred[i] != y_true[i]:
        print(f"Text: {val_texts[i]}")
        print(f"Actual: {'AI' if y_true[i] else 'Student'}")
        print(f"Predicted: {'AI' if y_pred[i] else 'Student'}\n")


In [23]:
def predict_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    pred = torch.argmax(outputs.logits).item()
    return "AI" if pred == 1 else "Student"


In [24]:
model.save_pretrained("llm-detector")
tokenizer.save_pretrained("llm-detector")


('llm-detector/tokenizer_config.json',
 'llm-detector/special_tokens_map.json',
 'llm-detector/vocab.txt',
 'llm-detector/added_tokens.json',
 'llm-detector/tokenizer.json')